In [11]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

In [12]:
import os
import pandas as pd
import numpy as np
import collections
import re
import string
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
os.environ['KERAS_BACKEND']='theano'
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
plt.switch_backend('agg')
from keras import backend as K
from tensorflow.keras.layers import Layer, InputSpec
import tensorflow as tf
from tensorflow import keras
from keras import initializers
%matplotlib inline

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
proj_dir='/content/drive/MyDrive/Colab Notebooks/doc2vec/' #give your project directory here. data sets should be in this location

In [15]:
'''this function is used to read text files.'''
def read_txt_file(file_name):
    with open(file_name,encoding="utf8", errors='ignore') as f:
        ###extract the boady of the text###
        line = f.readline()
        txt=''
        txt=txt+' '+line
        while line:
            line = f.readline()
            txt=txt+' '+line
        ###################################
    f.close()
    return(txt)

In [16]:
stop_words=set(stopwords.words('english'))#load stop words
punctuations=string.punctuation #get punctuations
lemmatizer = WordNetLemmatizer()
'''this function is used to clean text'''
def clean_txt(txt):
    txt=txt.lower() #set all characters to lowercase
    sentences=txt.split('\n')
    txt = ' '.join([i for i in sentences if not ':' in i])#remove headers
    txt = ''.join([i for i in txt if not i.isdigit()])#remove numbers

    ###remove urls and emails###
    words=txt.split()
    txt = ' '.join([i for i in words if not '@' in i and not '.com' in i and not  'http:' in i])
    #######################################

    ###remove punctuations###
    for character in punctuations:
        txt = txt.replace(character, '')
    #########################################
    
    ###remove stop words and lemmatize###
    words=txt.split()
    filtered_txt = ' '.join([lemmatizer.lemmatize(i) for i in words if not i in stop_words])
    #####################################
    
    return(filtered_txt)


In [17]:
def load_and_clean_data(location):    
    y=os.listdir(location)#get the list of folder
    txts=[]
    txts_cleaned=[]
    folder_array=[]
    file_array=[]
    for i in range(len(y)):
        text_file_names=os.listdir(location+'/'+y[i]) #get the list of files
        for text_file_name in text_file_names:
                file_array.append(text_file_name)
                txt=read_txt_file(location+'/'+y[i]+'/'+text_file_name) #read the text file
                txts.append(txt)
                txts_cleaned.append(clean_txt(txt)) #clean the text
                folder_array.append(y[i])

    ###create a data frame###
    df=pd.DataFrame()
    df['texts']=txts
    df['text cleaned']=txts_cleaned
    df['folder name']=folder_array
    df['file name']=file_array
    ########################
    return (df)


In [18]:
df_train=load_and_clean_data(proj_dir+'20news-bydate-train')
df_test=load_and_clean_data(proj_dir+'20news-bydate-test')


In [19]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

TOKENIZATION OF THE CLEANED TEXT

In [20]:
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(sen) for sen in df_train['text cleaned']]
tokenss = [word_tokenize(sen) for sen in df_test['text cleaned']]


CREATED ADDITION COLUMN FOR TOKENS


In [21]:
df_train['tokens']= tokens
df_test['tokenss']= tokenss


In [22]:
max_length = max([len(s.split()) for s in df_train['text cleaned']])
print('Maximum length: %d' % max_length)

Maximum length: 5890


In [23]:
data_train=df_train

In [24]:
data_test=df_test

CREATING THE VOCABULARY OF ALL WORDS

In [25]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

1385315 words total, with a vocabulary size of 76895
Max sentence length is 5892


In [26]:
all_test_words = [word for tokens in data_test["tokenss"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokenss"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

894440 words total, with a vocabulary size of 57615
Max sentence length is 5157


In [27]:
MAX_SEQUENCE_LENGTH = max([len(s.split()) for s in df_train['text cleaned']])
print('Maximum length: %d' % max_length)

Maximum length: 5890


In [28]:
MAX_SEQUENCE_LENGTH = 5890
EMBEDDING_DIM = 300

### Tokenize and Pad sequences

In [29]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["text cleaned"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["text cleaned"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))


Found 76897 unique tokens.


In [30]:
test_sequences = tokenizer.texts_to_sequences(data_test["text cleaned"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)


In [31]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

### Get Embeddings-Using GLOVE word2vec
GloVe is an unsupervised learning algorithm for obtaining vector representations for words

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

In [32]:
embeddings_index = {}
f = open('/content/glove.6B.300d.txt',encoding='utf8')# source of the downloaded glove files-use the 300d.txt
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors in Glove 6B 300d.' % len(embeddings_index))

Total 400000 word vectors in Glove 6B 300d.


In [33]:
embedding_matrix = np.random.random((len(train_word_index) + 1, EMBEDDING_DIM))
for word, i in train_word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [34]:
embedding_layer = Embedding(len(train_word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

**Predictor List**

In [39]:
class_names=df_train['folder name'].unique()
label_names=list(class_names)


In [40]:
y_train=(np.unique(df_train['folder name'], return_inverse=True)[1])

In [41]:
x_train = train_cnn_data
y_tr = y_train


Simple Neural Network without pre-training embeddings

In [36]:
import tensorflow as tf

In [37]:
from tensorflow.keras import layers

# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(len(train_word_index)+1, EMBEDDING_DIM)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a multiclass unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(len(label_names), activation="softmax", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with multiclass crossentropy loss and an rmsprop optimizer.
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print("Simple Neural Network")
model.summary()

Simple Neural Network
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5890)]            0         
                                                                 
 embedding_1 (Embedding)     (None, 5890, 300)         23069400  
                                                                 
 dropout (Dropout)           (None, 5890, 300)         0         
                                                                 
 conv1d (Conv1D)             (None, 1962, 128)         268928    
                                                                 
 conv1d_1 (Conv1D)           (None, 652, 128)          114816    
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                       

In [ ]:
epochs = 10

# Fit the model using the train and test datasets.
hist=model.fit(x_train, y_tr, validation_split=0.2, epochs=epochs)

First Model LSTM-Simple(takes time to run, dont know why)

In [42]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int64')
embedded_sequences = embedding_layer(sequence_input)
l_lstm = Bidirectional(LSTM(5))(embedded_sequences)
preds = Dense(len(label_names), activation='softmax')(l_lstm)
model = Model(sequence_input, preds)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("Bidirectional LSTM")
model.summary()

Bidirectional LSTM
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 5890)]            0         
                                                                 
 embedding (Embedding)       (None, 5890, 300)         23069400  
                                                                 
 bidirectional (Bidirectiona  (None, 10)               12240     
 l)                                                              
                                                                 
 dense_1 (Dense)             (None, 20)                220       
                                                                 
Total params: 23,081,860
Trainable params: 23,081,860
Non-trainable params: 0
_________________________________________________________________


**Training The Model**

In [44]:
epochs=30


In [ ]:
#cp=ModelCheckpoint('model_1.hdf5',monitor='val_acc',verbose=1,save_best_only=True)
history=model.fit(x_train, y_tr, validation_split=0.2,epochs=epochs, batch_size=20)#,callbacks=[cp])

Epoch 1/30
453/453 [==============================] - 396s 874ms/step - loss: 0.8493 - acc: 0.7627 - val_loss: 7.1346 - val_acc: 0.0601
Epoch 2/30
453/453 [==============================] - 395s 873ms/step - loss: 0.6914 - acc: 0.8084 - val_loss: 7.3120 - val_acc: 0.0742
Epoch 3/30
453/453 [==============================] - 396s 874ms/step - loss: 0.5756 - acc: 0.8360 - val_loss: 7.7305 - val_acc: 0.0959
Epoch 4/30
453/453 [==============================] - 398s 879ms/step - loss: 0.4943 - acc: 0.8548 - val_loss: 8.0169 - val_acc: 0.0893
Epoch 5/30
453/453 [==============================] - 398s 879ms/step - loss: 0.4298 - acc: 0.8745 - val_loss: 8.1832 - val_acc: 0.0937
Epoch 6/30
453/453 [==============================] - 398s 879ms/step - loss: 0.3813 - acc: 0.8876 - val_loss: 8.4028 - val_acc: 0.0972
Epoch 7/30
453/453 [==============================] - 397s 876ms/step - loss: 0.2399 - acc: 0.9486 - val_loss: 8.8892 - val_acc: 0.0924
Epoch 10/30
453/453 [===========================

Second CNN MODEL- We need to discuss ( Runs fine By the validation and accuracy is not showing properly

In [ ]:
from tensorflow.keras import layers

def ConvNet1(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
    x = layers.MaxPooling1D(5)(x)
    x = layers.Conv1D(128, 5, activation="relu")(x)
    x = layers.MaxPooling1D(5)(x)
    x = layers.Conv1D(128, 5, activation="relu")(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    preds = Dense(labels_index, activation='softmax')(x)
    model = keras.Model(sequence_input, preds)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer="adam",
                  metrics=['acc'])
    model.summary()
    return model

In [ ]:
class_names=df_train['folder name'].unique()
label_names=list(class_names)
len(label_names)


In [ ]:
y_train=(np.unique(df_train['folder name'], return_inverse=True)[1])
y_train

In [ ]:
x_train = train_cnn_data
y_tr = y_train



In [ ]:
model1 = ConvNet1(embedding_matrix, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(label_names))

In [ ]:
num_epochs = 100
batch_size = 34

In [ ]:
import numpy as np
x_train = np.random.rand(10,5890) #(batch_size, input_shape)
y_tr = np.random.rand(10)
y_tr=y_tr.astype(int)

Training The second Model(CNN Model)

In [ ]:
hist = model1.fit(x_train, y_tr, epochs=num_epochs, shuffle=True,validation_split=0.2, batch_size=batch_size)

Another CNN Model with different Kernels

In [ ]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=500, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Flatten()(x) 
    preds = Dense(labels_index, activation='softmax')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
# = data_train['folder name']
class_names=df_train['folder name'].unique()
label_names=list(class_names)
len(label_names)

In [ ]:
y_train=(np.unique(df_train['folder name'], return_inverse=True)[1])
y_train

In [ ]:
x_train = train_cnn_data
y_train=(np.unique(df_train['folder name'], return_inverse=True)[1])
y_tr=y_train
y_tr=y_tr.astype(int)

In [ ]:
model2 = ConvNet(embedding_matrix, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM,len(label_names))

Training the second CNN with Kernel model

In [ ]:
num_epochs = 30
batch_size = 34

In [ ]:
import numpy as np
x_train = np.random.rand(10,5890) #(batch_size, input_shape)
y_tr = np.random.rand(10)
y_tr=y_tr.astype(int)

In [ ]:
hist = model2.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.2, shuffle=True, batch_size=batch_size)

Testing Part-Work in Progress

In [ ]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)